## Decision Tree

In [1]:
import pandas as pd
from source.DecisionTree import get_gain, get_probability, get_entropy, build_decision_tree, get_fmax

In [3]:
file = './datasets/DecisionTree'
dat = pd.read_csv(file, sep='\t')
# Get label and features
label = 'Mode'
features = (dat.drop(label, axis=1)).keys()
# Get the gain
a = get_gain(dat,'Mode','Gender')
print(f'Gain(S,Gender) = {a:.3}')
# Print max feature
fmax = get_fmax(dat, label)
print(f'The max feature is "{fmax}"')

Gain(S,Gender) = 0.125
The max feature is "Travel Cost"


In [4]:
# Result
build_decision_tree(dat, label)

{'Travel Cost': {'Cheap': {'Gender': {'Male': 'Bus',
    'Female': {'Ownership': {0: 'Bus', 1: 'Train'}}}},
  'Expensive': 'Car',
  'Standard': 'Train'}}

## NaiveBayes

### Features Selection

In [47]:
file_train = './datasets/Titanic_train.csv'
file_test = './datasets/Titanic_test.csv'
X_train = pd.read_csv(file_train, index_col=0)
X_test = pd.read_csv(file_test)
features = ['Sex', 'Embarked' ,'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
categories = ['Sex', 'Embarked']
drops = ['Cabin', 'Ticket', 'Name']
X_train.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [48]:
X_train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


### Pre-Processing

In [51]:
import pandas as pd
from source.Sampling import categorical_to_numberic, smote
from source.NaiveBayes import naive_bayes
# Convert all the categories to numberic
X_train = categorical_to_numberic(X_train, categories)
X_test = categorical_to_numberic(X_test, categories)
# Drop all the record that has the missing value in Feature EMBARKED and FARE
X_train = X_train.dropna(subset=['Embarked'], how='any')
X_test = X_test.dropna(subset=['Fare'], how='any')
idx_test = X_test['PassengerId']
# Fill the missing value with mean value in the table
X_train['Age'] = X_train['Age'].fillna(value=X_train['Age'].mean())
X_test['Age'] = X_test['Age'].fillna(value=X_test['Age'].mean())

In [50]:
# Checking missing import features
X_train[features].isnull().sum()

Sex           0
Embarked      2
Pclass        0
Age         177
SibSp         0
Parch         0
Fare          0
dtype: int64

In [43]:
X_train, y_train = smote(X_train.drop(drops, axis=1), 'Survived')

### Building BayesModel

In [46]:
model = naive_bayes(X_train, y_train)

NameError: name 'naive_bayes' is not defined

In [50]:
# predict using test data
y_pred = model.predict(X_test)
print(y_pred)

[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 0 1 1 1 0 1
 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 1
 1 1 0 1 0 1 0 1 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0
 0 0 1 0 1 0 0 1 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 1 0 1 0
 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1 1 0 1 0 0 0 0 1 0 0 1 1 1 0 1 0 1
 0 1 1 0 1 0 0 0 1 0 0 1 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1 0
 0 0 1 1 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 0 1 0 0 1
 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 1
 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0 0
 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 1 0 0 1
 1 1 1 1 1 0 1 0 0 0]


In [51]:
# Compute predicted probabilities: y_pred_prob
y_pred_prob = model.predict_proba(X_test)

In [52]:
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score
print('Confusion matrix:\n', confusion_matrix(y_test,y_pred)) 
print("Accuracy: ", accuracy_score(y_test,y_pred))

Confusion matrix:
 [[228  37]
 [  7 145]]
Accuracy:  0.894484412470024


In [60]:
# predict using test data
y_pred = model.predict(X_test[0:5])
y_pred_prob = model.predict_proba(X_test[0:5])
print(y_pred)
print(y_pred_prob)

[0 1 0 0 1]
[[0.90819901 0.09180099]
 [0.2918426  0.7081574 ]
 [0.7832132  0.2167868 ]
 [0.9353446  0.0646554 ]
 [0.23384532 0.76615468]]


In [5]:
import pandas as pd
from source.Sampling import categorical_to_numberic, smote

file_train = './datasets/Titanic_train.csv'
file_test = './datasets/Titanic_test.csv'
X_train = pd.read_csv(file_train, index_col=0)
X_test = pd.read_csv(file_test)
# features = ['Sex', 'Embarked' ,'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
categories = ['Sex', 'Embarked']
drops = ['Cabin', 'Ticket', 'Name']
X_train.describe()
X_train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [3]:
X = categorical_to_numberic(X_train,categories)
X

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0
5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",0,27.0,0,0,211536,13.0000,NaN,0
888,1,1,"Graham, Miss. Margaret Edith",1,19.0,0,0,112053,30.0000,B42,0
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,NaN,1,2,W./C. 6607,23.4500,NaN,0


In [20]:
# Drop all the record that has the missing value in Feature EMBARKED and FARE
# X = X.dropna(subset=['Embarked'], how='any')
# X = X.dropna(subset=['Fare'], how='any')
X_dat,Xo = smote(X.drop(drops, axis=1),'Survived')
pd.DataFrame(X_dat)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,47.000000,1,0,7.250000,0
1,1,1,62.000000,1,0,71.283300,1
2,3,1,27.000000,0,0,7.925000,0
3,1,1,22.000000,1,0,53.100000,0
4,3,0,14.000000,0,0,8.050000,0
...,...,...,...,...,...,...,...
1093,1,1,30.170887,0,0,67.670842,0
1094,1,0,30.366563,1,0,55.791838,0
1095,1,1,30.170887,1,0,39.459913,0
1096,2,0,30.170887,1,1,19.234171,0


In [28]:
print(Xo.loc[Xo == 1].count())
print(Xo.loc[Xo == 0].count())

549
549


In [16]:
X.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age           0
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      0
dtype: int64

In [15]:
X['Age'] = X['Age'].fillna(value=X['Age'].mean())